In [0]:

%pip install loguru==0.7.2
%pip install hydra-core==1.3
%pip install python-dotenv==1.0.1
%pip install numpy==1.24.4
%pip install cryptography==43.0.1
%pip install gensim==4.3.3
%pip install cython==3.0.11
%pip install spacy==3.4.4 #3.0.4
%pip install thinc==8.1.7
%pip install pandas==2.0.0
%pip install snowflake-connector-python==3.12.2
%pip install transformers==4.46.1
%pip install pyarrow==16.0.0
%pip install datasets==3.1.0
%pip install evaluate==0.4.3
%pip install pyspark==3.5.3
%pip install dask==2023.10.1 
%pip install distributed==2023.10.1
%pip install torch==2.0.0
# %pip install cymem==2.0.8
# %pip install scikit-learn==1.1.0
# %pip install typer==0.7.0
# %pip install accelerate==0.26.0

1


In [0]:
dbutils.library.restartPython()

2024-12-13 14:59:39 | DEBUG   | text_preprocessing.py:197 | Tokenized document into 1 tokens.
2024-12-13 14:59:39 | DEBUG   | text_preprocessing.py:197 | Tokenized document into 0 tokens.


In [0]:
# import time 

# while 1>0:
#   time.sleep(60)

In [0]:
import ast
import numpy as np
import pandas as pd
import gc
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F

from centralized_nlp_package.data_access import (
    read_from_snowflake,
    write_dataframe_to_snowflake
)
from centralized_nlp_package.data_processing import (
  check_pd_dataframe_for_records,
    initialize_dask_client,
    df_apply_transformations,
    dask_compute_with_progress,
    pandas_to_spark,
    convert_columns_to_timestamp
)
from centralized_nlp_package.text_processing import (initialize_spacy, get_match_set)

from topic_modelling_package.reports import create_topic_dict, generate_topic_report, replace_separator_in_dict_words

/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized_nlp_package/utils/config.py:75: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize(config_path="../configs"):


2024-12-13 14:59:57 | INFO    | logging_setup.py:62 | Logging setup completed.


/databricks/python/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/topic_modelling_package/utils/config.py:15: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize(config_path="../config"):


2024-12-13 15:00:11 | INFO    | logging.py:45 | Logging setup completed.


## Load dataset

In [0]:

tsQuery= ("SELECT CALL_ID,ENTITY_ID, DATE, FILT_MD, FILT_QA, SENT_LABELS_FILT_MD,"
         "SENT_LABELS_FILT_QA, CALL_NAME,COMPANY_NAME,EARNINGS_CALL,ERROR,TRANSCRIPT_STATUS,UPLOAD_DT_UTC,VERSION_ID,EVENT_DATETIME_UTC,PARSED_DATETIME_EASTERN_TZ "
          "FROM EDS_PROD.QUANT_LIVE.CTS_FUND_COMBINED_SCORES_H  t2  ORDER BY PARSED_DATETIME_EASTERN_TZ DESC LIMIT 10")
          



In [0]:
resultspkdf = read_from_snowflake(tsQuery)

2024-12-13 15:00:13 | INFO    | snowflake_utils.py:226 | Reading data from Snowflake using Spark.
Spark session already initialized; reusing existing session.
Spark session already initialized; reusing existing session.
2024-12-13 15:00:13 | INFO    | helper.py:27 | Environment provided by user: quant
2024-12-13 15:00:14 | DEBUG   | snowflake_utils.py:123 | Retrieved secrets from AKV successfully.
2024-12-13 15:00:14 | DEBUG   | snowflake_utils.py:135 | Private key loaded successfully.
2024-12-13 15:00:14 | DEBUG   | snowflake_utils.py:147 | Private key serialized to PEM format.
2024-12-13 15:00:14 | DEBUG   | snowflake_utils.py:155 | Private key decoded and cleaned.
2024-12-13 15:00:14 | DEBUG   | snowflake_utils.py:190 | Snowflake connection options constructed.
2024-12-13 15:00:14 | DEBUG   | snowflake_utils.py:231 | Executing query: SELECT CALL_ID,ENTITY_ID, DATE, FILT_MD, FILT_QA, SENT_LABELS_FILT_MD,SENT_LABELS_FILT_QA, CALL_NAME,COMPANY_NAME,EARNINGS_CALL,ERROR,TRANSCRIPT_STATUS

In [0]:
currdf_old = resultspkdf.toPandas()
currdf_ref = currdf_old.copy()

/databricks/spark/python/pyspark/sql/pandas/utils.py:105: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [CALL_ID, EARNINGS_CALL] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
  warnings.warn(


### Checking records

### old

In [0]:
import os
if len(currdf_old)>0:
    print('The data spans from ' + str(currdf_old['PARSED_DATETIME_EASTERN_TZ'].min()) + ' to ' + str(currdf_old['PARSED_DATETIME_EASTERN_TZ'].max()) + 'and has ' + str(currdf_old.shape[0]) + ' rows and ' + str(currdf_old.shape[1]) + ' columns.')
else:
    print('No new transcripts to parse.')
    dbutils.notebook.exit(1)
    os._exit(1)

The data spans from 2024-12-13 01:14:51 to 2024-12-13 01:14:51and has 10 rows and 16 columns.


#### Reafctored

In [0]:
check_pd_dataframe_for_records(currdf_old, datetime_col = 'PARSED_DATETIME_EASTERN_TZ')

2024-12-13 15:00:57 | INFO    | dataframe_utils.py:230 | The data spans from 2024-12-13 01:14:51 to 2024-12-13 01:14:51 and has 10 rows and 16 columns.


### Apply Transformation

### old

This code processes a DataFrame named currdf_old by converting the CALL_ID column to strings and parsing string representations of lists in several columns into actual lists using ast.literal_eval. It calculates the lengths of the lists in the FILT_MD and FILT_QA columns, storing these lengths in new columns LEN_FILT_MD and LEN_FILT_QA. The code then filters out sentences in the FILT_QA column that end with a question mark, updating both the FILT_QA and SENT_LABELS_FILT_QA columns accordingly. This transformation is part of a workflow to prepare and validate data by comparing outputs from the original and refactored code implementations.



In [0]:

currdf_old['CALL_ID'] = currdf_old['CALL_ID'].apply(lambda x: str(x))
currdf_old['FILT_MD'] = currdf_old['FILT_MD'].apply(ast.literal_eval)
currdf_old['FILT_QA'] = currdf_old['FILT_QA'].apply(ast.literal_eval)
currdf_old['SENT_LABELS_FILT_MD'] = currdf_old['SENT_LABELS_FILT_MD'].apply(ast.literal_eval)
currdf_old['SENT_LABELS_FILT_QA'] = currdf_old['SENT_LABELS_FILT_QA'].apply(ast.literal_eval)
currdf_old['LEN_FILT_MD'] = currdf_old['FILT_MD'].apply(len)
currdf_old['LEN_FILT_QA'] = currdf_old['FILT_QA'].apply(len)
currdf_old['SENT_LABELS_FILT_QA'] = currdf_old.apply(lambda x: [x['SENT_LABELS_FILT_QA'][i] for i, sent in enumerate(x['FILT_QA']) if not sent.endswith('?')], axis=1)
currdf_old['FILT_QA'] = currdf_old['FILT_QA'].apply(lambda x: [sent for sent in x if not sent.endswith('?')])


currdf_old.head(5)

,CALL_ID,ENTITY_ID,DATE,FILT_MD,FILT_QA,SENT_LABELS_FILT_MD,SENT_LABELS_FILT_QA,CALL_NAME,COMPANY_NAME,EARNINGS_CALL,ERROR,TRANSCRIPT_STATUS,UPLOAD_DT_UTC,VERSION_ID,EVENT_DATETIME_UTC,PARSED_DATETIME_EASTERN_TZ,LEN_FILT_MD,LEN_FILT_QA
0,3133014,000CCP-E,2024-12-12,[Let me start by addressing our third quarter ...,[Sorry if I missed this in the earlier comment...,"[0, -1, -1, 1, -1, -1, 1, 1, 1, 0, 1, -1, 0, 0...","[0, 0, -1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0...",Q3 2025 Earnings Call,"AstroNova, Inc.",1,no,CorrectedTranscript,2024-12-12 16:55:56,7661968,2024-12-12 14:00:00,2024-12-13 01:14:51,96,40
1,3145024,05HFQP-E,2024-12-11,"[It's my pleasure to join you today., In the i...","[ for taking my question., I just wondered if ...","[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...",Q3 2024 Earnings Call,Industria de Diseño Textil SA,1,no,CorrectedTranscript,2024-12-12 20:29:24,7662284,2024-12-11 00:00:00,2024-12-13 01:14:51,100,86
2,3142307,0GYVRX-E,2024-12-12,[I'll start by sharing a high-level overview o...,"[First, I just wanted to ask about your Q4 out...","[0, 0, 0, 0, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",Q3 2025 Earnings Call,The Lovesac Co.,1,no,CorrectedTranscript,2024-12-12 17:58:40,7662053,2024-12-12 13:30:00,2024-12-13 01:14:51,164,205
3,3130706,000RCZ-E,2024-12-12,[ for joining Nordson's fiscal 2024 fourth qua...,[Appreciate all the thoughts at the end there ...,"[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, -1, 1, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, ...",Q4 2024 Earnings Call,Nordson Corp.,1,no,CorrectedTranscript,2024-12-12 18:35:23,7662109,2024-12-12 13:30:00,2024-12-13 01:14:51,140,248
4,3138897,003NFW-E,2024-12-12,[We posted solid results in Q4 and I thank our...,"[We're seeing a lot of consolidation activity,...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, ...","[0, 0, -1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0...",Q4 2024 Earnings Call,TC Transcontinental,1,no,CorrectedTranscript,2024-12-12 23:26:10,7662405,2024-12-12 13:00:00,2024-12-13 01:14:51,95,153


## Refactored

In [0]:
## cleaner, easliy readble and understandable

col_inti_tranform = [
  ("CALL_ID","CALL_ID", str),
    ("FILT_MD", "FILT_MD", ast.literal_eval),
    ("FILT_QA", "FILT_QA", ast.literal_eval),
    ("SENT_LABELS_FILT_QA", "SENT_LABELS_FILT_QA", ast.literal_eval),
    ("SENT_LABELS_FILT_MD", "SENT_LABELS_FILT_MD", ast.literal_eval),
    ('LEN_FILT_MD', 'FILT_MD', len),
    ('LEN_FILT_QA', 'FILT_QA', len),
    ('SENT_LABELS_FILT_QA', ['SENT_LABELS_FILT_QA','FILT_QA'], (lambda x: [x['SENT_LABELS_FILT_QA'][i] 
                                                                           for i, sent in enumerate(x['FILT_QA']) 
                                                                           if not sent.endswith('?')])),
    ('FILT_QA', 'FILT_QA', lambda x: [sent for sent in x if not sent.endswith('?')])
]

currdf_ref = df_apply_transformations(currdf_ref, col_inti_tranform)
print("The DataFrames are not identical.")
currdf_ref.head(5)

2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'CALL_ID' to create 'CALL_ID'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'CALL_ID'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'FILT_MD' to create 'FILT_MD'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'FILT_MD'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'FILT_QA' to create 'FILT_QA'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'FILT_QA'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'SENT_LABELS_FILT_QA' to create 'SENT_LABELS_FILT_QA'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'SENT_LABELS_FILT_QA'.
The DataFrames a

,CALL_ID,ENTITY_ID,DATE,FILT_MD,FILT_QA,SENT_LABELS_FILT_MD,SENT_LABELS_FILT_QA,CALL_NAME,COMPANY_NAME,EARNINGS_CALL,ERROR,TRANSCRIPT_STATUS,UPLOAD_DT_UTC,VERSION_ID,EVENT_DATETIME_UTC,PARSED_DATETIME_EASTERN_TZ,LEN_FILT_MD,LEN_FILT_QA
0,3133014,000CCP-E,2024-12-12,[Let me start by addressing our third quarter ...,[Sorry if I missed this in the earlier comment...,"[0, -1, -1, 1, -1, -1, 1, 1, 1, 0, 1, -1, 0, 0...","[0, 0, -1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0...",Q3 2025 Earnings Call,"AstroNova, Inc.",1,no,CorrectedTranscript,2024-12-12 16:55:56,7661968,2024-12-12 14:00:00,2024-12-13 01:14:51,96,40
1,3145024,05HFQP-E,2024-12-11,"[It's my pleasure to join you today., In the i...","[ for taking my question., I just wondered if ...","[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...",Q3 2024 Earnings Call,Industria de Diseño Textil SA,1,no,CorrectedTranscript,2024-12-12 20:29:24,7662284,2024-12-11 00:00:00,2024-12-13 01:14:51,100,86
2,3142307,0GYVRX-E,2024-12-12,[I'll start by sharing a high-level overview o...,"[First, I just wanted to ask about your Q4 out...","[0, 0, 0, 0, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",Q3 2025 Earnings Call,The Lovesac Co.,1,no,CorrectedTranscript,2024-12-12 17:58:40,7662053,2024-12-12 13:30:00,2024-12-13 01:14:51,164,205
3,3130706,000RCZ-E,2024-12-12,[ for joining Nordson's fiscal 2024 fourth qua...,[Appreciate all the thoughts at the end there ...,"[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, -1, 1, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, ...",Q4 2024 Earnings Call,Nordson Corp.,1,no,CorrectedTranscript,2024-12-12 18:35:23,7662109,2024-12-12 13:30:00,2024-12-13 01:14:51,140,248
4,3138897,003NFW-E,2024-12-12,[We posted solid results in Q4 and I thank our...,"[We're seeing a lot of consolidation activity,...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, ...","[0, 0, -1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0...",Q4 2024 Earnings Call,TC Transcontinental,1,no,CorrectedTranscript,2024-12-12 23:26:10,7662405,2024-12-12 13:00:00,2024-12-13 01:14:51,95,153


2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'LEN_FILT_MD'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'FILT_QA' to create 'LEN_FILT_QA'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'LEN_FILT_QA'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:118 | Applying transformation on multiple columns ['SENT_LABELS_FILT_QA', 'FILT_QA'] to create 'SENT_LABELS_FILT_QA'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'SENT_LABELS_FILT_QA'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'FILT_QA' to create 'FILT_QA'.
2024-12-13 15:00:57 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'FILT_QA'.
2024-12-13 15:00:57 | INFO    | dataframe_utils.py:132 | All transformations applied successfully.


In [0]:
assert currdf_old.equals(currdf_ref), "The DataFrames are not identical."

### Spacy model initalization

Intialize spacy object

#### old 

In [0]:
%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0.tar.gz

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 27.8 MB/s eta 0:00:00
  Created wheel for en-core-web-sm: filename=en_core_web_sm-3.5.0-py3-none-any.whl size=12803284 sha256=1b6668a965840c7018921b95adc67276a7ee7c416d5926723d470e961e287294
  Stored in directory: /root/.cache/pip/wheels/88/70/a5/ea06bb10f3fe2c6bbfa0e36e3deca1adbf3648228490b2925c
Successfully built en-core-web-sm
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.7
    Uninstalling thinc-8.1.7:
      Successfully uninstalled thinc-8.1.7
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spa

In [0]:
import spacy 
nlp = spacy.load("en_core_web_sm", disable = ['parser'])
# Excluding financially reavant stopwords
nlp.Defaults.stop_words -= {"bottom", "top", "Bottom", "Top", "call", "down"}
nlp.max_length = 1000000000

### refactored

In [0]:
nlp_ref = initialize_spacy()

2024-12-13 15:01:37 | INFO    | text_preprocessing.py:55 | Loading SpaCy model: en_core_web_sm
2024-12-13 15:01:38 | DEBUG   | text_preprocessing.py:62 | Excluded stop words: ['bottom', 'top', 'Bottom', 'Top', 'call']
2024-12-13 15:01:38 | INFO    | text_preprocessing.py:64 | SpaCy model initialized successfully.


### Load Topics dictionary

Loading topics 

In [0]:
import pandas as pd

client ='MASS'
version = 'v0'
use_case = ''
match_df_v0_old = pd.read_csv(f"/dbfs/mnt/access_work/UC25/Topic Modeling/NLI Models/{client}_{use_case}_dictionaries_{version}.csv")
match_df_v0_ref = match_df_v0_old.copy()

### Match keyword list explosion

#### old

In [0]:
match_df_v0_old['Refined Keywords'] = match_df_v0_old['Refined Keywords'].apply(ast.literal_eval)
match_df_old = match_df_v0_old[['Subtopic','Refined Keywords']].explode(column='Refined Keywords')

match_df_old.head(5)

,Subtopic,Refined Keywords
0,Job Creation,recruit_aggressively
0,Job Creation,hire_salesperson
0,Job Creation,hire_rep
0,Job Creation,hire_engineer
0,Job Creation,hire_aggressively


#### ref

In [0]:
match_df_v0_ref = df_apply_transformations(match_df_v0_ref, [('Refined Keywords', 'Refined Keywords', ast.literal_eval)])
match_df_ref = match_df_v0_ref[['Subtopic','Refined Keywords']].explode(column='Refined Keywords')

match_df_ref.head(5)

2024-12-13 15:01:08 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'Refined Keywords' to create 'Refined Keywords'.
2024-12-13 15:01:08 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'Refined Keywords'.


,Subtopic,Refined Keywords
0,Job Creation,recruit_aggressively
0,Job Creation,hire_salesperson
0,Job Creation,hire_rep
0,Job Creation,hire_engineer
0,Job Creation,hire_aggressively


2024-12-13 15:01:08 | INFO    | dataframe_utils.py:132 | All transformations applied successfully.


In [0]:
assert match_df_old.equals(match_df_ref), "The DataFrames are not identical."

### Common code

This process involves filtering and transforming data to handle negation logic within a dataset. It begins by isolating rows with non-null negation information, then parses and expands these entries to ensure each negation term is treated individually. A flag is added to indicate the presence of negation, and the data is reorganized to align with a standard format. Finally, the processed negation data is combined with the main dataset, integrating both negation and refined keyword information for comprehensive analysis.



In [0]:
match_df_negate = match_df_v0_ref[~match_df_v0_ref['Negation'].isna()][['Subtopic', 'Negation']]#.apply(lambda x: ast.literal_eval(x['Negation']), axis=1)#.explode(column = 'Negation')
match_df_negate = df_apply_transformations(match_df_negate, [('Negation', 'Negation', ast.literal_eval)])
match_df_negate = match_df_negate.explode(column = 'Negation')
match_df_negate['negate'] = True
match_df_negate = match_df_negate.rename(columns = {'Subtopic': 'label', 'Negation': 'match'})
match_df_ref['negate'] = False
match_df_ref = match_df_ref.rename(columns={'Subtopic':'label', 'Refined Keywords':'match'})
match_df_ref = pd.concat([match_df_ref, match_df_negate])

2024-12-13 15:01:13 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'Negation' to create 'Negation'.
2024-12-13 15:01:13 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'Negation'.
2024-12-13 15:01:13 | INFO    | dataframe_utils.py:132 | All transformations applied successfully.


### create topic dictionary

This process involves tokenizing and categorizing words from a dataset into different sets based on their linguistic properties and context. The matchTokenize function processes text to identify and lower-case proper nouns and significant words, excluding stop words, punctuation, and numbers. The get_match_set_old function then organizes these tokens into unigrams, bigrams, and phrases, creating distinct sets for each based on their structure and composition. Finally, the code constructs dictionaries to map topics to their corresponding word sets and negation terms, ensuring that each topic's keywords and negations are systematically categorized for further analysis.



#### old

In [0]:
match_df_old = match_df_ref.copy()

In [0]:
def matchTokenize(doc):
  ret = []
  for ent in nlp(doc):
    if ent.pos_ == 'PROPN' or ent.text[0].isupper():
      ret.append(ent.text.lower())
    #  print(ent.text.lower())
    #  print(ent.text)
      continue
    if not ent.is_stop and not ent.is_punct and ent.pos_ != 'NUM':
      ret.append(ent.lemma_.lower())
  return ret

def get_match_set_old(matches):
  
  bigrams = set([word.lower() for word in matches if len(word.split('_'))==2] + [word.lower().replace(" ", '_') for word in matches if len(word.split(' '))==2] + ['_'.join(matchTokenize(word)) for word in matches if len(word.split(' '))==2])
 
  unigrams = set([matchTokenize(match)[0] for match in matches if ('_' not in match) and (len(match.split(' '))==1)] + [match.lower() for match in matches if ('_' not in match) and (len(match.split(' '))==1)])

  # phrases = set([phrase.lower() for phrase in matches if len(phrase.split(" "))>2] + [' '.join(matchTokenize(phrase)) for phrase in matches if len(phrase.split(" "))>2])

  # Phrase matching correction
  phrases = [phrase.lower() for phrase in matches if len(phrase.split(" "))>2]
  
  #print(matches)
  #print(unigrams, bigrams, phrases)
  return {'original': matches, 'unigrams': unigrams, 'bigrams': bigrams, 'phrases': phrases}
  


# COMMAND ----------

word_set_dict_old = {topic.replace(' ', '_').upper() : get_match_set_old(match_df_old[(match_df_old['label']==topic) & (match_df_old['negate']==False)]['match'].values) for topic in match_df_old['label'].unique()}

negate_dict_old = {topic.replace(' ', '_').upper() : [word.lower() for word in match_df_old[(match_df_old['label']==topic) & (match_df_old['negate']==True)]['match'].values.tolist()] for topic in match_df_old['label'].unique()}


In [0]:
word_set_dict_old = {topic.replace(' ', '_').upper() : get_match_set_old(match_df_old[(match_df_old['label']==topic) & (match_df_old['negate']==False)]['match'].values) for topic in match_df_old['label'].unique()}

negate_dict_old = {topic.replace(' ', '_').upper() : [word.lower() for word in match_df_old[(match_df_old['label']==topic) & (match_df_old['negate']==True)]['match'].values.tolist()] for topic in match_df_old['label'].unique()}


#### Refactored

In [0]:
word_set_dict_ref, negate_dict_ref = create_topic_dict(match_df_ref, nlp_ref)

2024-12-13 15:01:49 | DEBUG   | text_preprocessing.py:225 | Tokenized matched words into 1 tokens.
2024-12-13 15:01:49 | DEBUG   | text_preprocessing.py:225 | Tokenized matched words into 1 tokens.
2024-12-13 15:01:49 | DEBUG   | match_operations.py:78 | Generated match patterns: {'original': ['recruit_aggressively', 'hire_salesperson', 'hire_rep', 'hire_engineer', 'hire_aggressively', 'hire_onboarde', 'recruiting_effort', 'add_people', 'expand_workforce', 'actively_recruit', 'hire_people', 'recruit_hire', 'recruit', 'aggressive_hiring', 'hire'], 'unigrams': {'recruit', 'hire'}, 'bigrams': {'recruit_aggressively', 'actively_recruit', 'aggressive_hiring', 'recruit_hire', 'add_people', 'hire_aggressively', 'hire_rep', 'hire_onboarde', 'recruiting_effort', 'expand_workforce', 'hire_salesperson', 'hire_people', 'hire_engineer'}, 'phrases': []}
2024-12-13 15:01:49 | DEBUG   | report_generation.py:82 | Processed topic 'JOB_CREATION': 15 positive matches, 45 negative matches.
2024-12-13 15:01

In [0]:
assert all(
    key in word_set_dict_ref and
    all(np.array_equal(word_set_dict_old[key][sub_key], word_set_dict_ref[key][sub_key])
        for sub_key in word_set_dict_old[key])
    for key in word_set_dict_old
), "The dictionaries are not equal."

### Replace separator in dicts with underscore

This process refines a dictionary of negation terms by converting bigrams (words with underscores) into space-separated phrases, ensuring consistent formatting across all entries.



#### old 

In [0]:
negate_dict1_old = {k: [] for k in negate_dict_old.keys()}
for k, v in negate_dict_old.items():
  for word in v:
    if len(word.split('_'))==2:
      new_word = ' '.join(word.split('_'))
      negate_dict1_old[k].append(new_word)
    else:
      negate_dict1_old[k].append(word)

#### refactored

In [0]:
negate_dict1_ref = replace_separator_in_dict_words(negate_dict_ref)

2024-12-13 15:01:58 | DEBUG   | report_generation.py:144 | No transformation for 'not hiring' in category 'JOB_CREATION'.
2024-12-13 15:01:58 | DEBUG   | report_generation.py:144 | No transformation for 'not hire' in category 'JOB_CREATION'.
2024-12-13 15:01:58 | DEBUG   | report_generation.py:144 | No transformation for 'reduce additional hiring' in category 'JOB_CREATION'.
2024-12-13 15:01:58 | DEBUG   | report_generation.py:141 | Transformed 'hiring_freeze' to 'hiring freeze' in category 'JOB_CREATION'.
2024-12-13 15:01:58 | DEBUG   | report_generation.py:141 | Transformed 'freeze_hire' to 'freeze hire' in category 'JOB_CREATION'.
2024-12-13 15:01:58 | DEBUG   | report_generation.py:141 | Transformed 'recruitment_freeze' to 'recruitment freeze' in category 'JOB_CREATION'.
2024-12-13 15:01:58 | DEBUG   | report_generation.py:141 | Transformed 'freeze_hiring' to 'freeze hiring' in category 'JOB_CREATION'.
2024-12-13 15:01:58 | DEBUG   | report_generation.py:141 | Transformed 'slow_hir

In [0]:
assert all(np.array_equal(negate_dict1_old[key], negate_dict1_ref[key]) for key in negate_dict1_ref), "The dictionaries are not equal."

### Genearte Topic report 

This code processes text data to identify and count occurrences of specific words and phrases, using tokenization and n-gram analysis. It begins by tokenizing text into unigrams and bigrams, then matches these against predefined sets of keywords and phrases, while considering negation terms to suppress certain matches. The results are stored in a DataFrame, where each topic's total and statistical match counts are calculated. Additional statistics, such as relevance and sentiment scores, are derived from these counts to provide insights into the text's content and sentiment. The process leverages Dask for parallel computation to efficiently handle large datasets.



#### old

In [0]:
def wordTokenize(doc):
  
  return [ent.lemma_.lower() for ent in nlp(doc) if not ent.is_stop and not ent.is_punct and ent.pos_ != 'NUM']

def find_ngrams(input_list, n):
  return zip(*[input_list[i:] for i in range(n)])



def match_count_lowStat(texts, match_sets, phrases = True, suppress = None):

  count_dict = {label : {matchw: 0 for matchw in match_set['unigrams'].union(match_set['bigrams']) } for label, match_set in match_sets.items()}
  total_counts = {label: [] for label in match_sets.keys()}

  for text in texts:
    
    counted = {label: 0 for label in match_sets.keys()}
    unigrams = wordTokenize(text)
    bigrams = ['_'.join(g) for g in find_ngrams(unigrams, 2)]
    
    text = text.lower()
    for label, match_set in match_sets.items(): 
      
      if any(item in text for item in suppress[label]):
        counted[label] += 0
        continue
        
      for word in unigrams:
        if word in match_set['unigrams']:
          count_dict[label][word]+=1
          counted[label] += 1

      for word in bigrams:
        if word in match_set['bigrams']:
          count_dict[label][word]+=1
          counted[label] += 1
      
      if phrases:
        if any(phrase in text for phrase in match_set['phrases']):
          counted[label] += 1
          continue

    for label in match_sets.keys():
      
      total_counts[label].append(counted[label])

    
  return {label : {'total': total_counts[label], 'stats' : count_dict[label]} for label in match_sets.keys()}

def sentscore(a, b, weight = True):
  
  # number of relevant sentences
      
  length = len(a)
  if length==0:
      return None
  if length!=len(b):
      return None
  num = len([x for x in a if x>0])
  
  if num==0:
      return None
   
  if weight==True:
    return np.dot(a,b)/num
  else:
    return np.dot([1 if x>0 else 0 for x in a], b)/num

In [0]:
currdf_old = dd.from_pandas(currdf_old.head(2), npartitions = 4)
for label, section in {'FILT_MD': 'FILT_MD', 'FILT_QA': 'FILT_QA'}.items():

  currdf_old['matches_' + label] = currdf_old[section].apply(lambda x: match_count_lowStat(x, word_set_dict_old, phrases = True, suppress = negate_dict1_old), meta = ('matches_' + label, object))
  #currdf_old[label] = currdf_old['matches_' + label].apply(lambda x: [str(calc['filt']) for calc in x], meta = ('FILT_' + label, object))

# Running Dask compute
with ProgressBar():
  currdf_old = currdf_old.compute()


for label, section in {'FILT_MD': 'MGNT_DISCUSSION', 'FILT_QA': 'QA_SECTION'}.items():
  currdf_old['matches_' + label] = currdf_old['matches_' + label].apply(ast.literal_eval)
  for topic in word_set_dict_old.keys():
    
    currdf_old[topic + '_TOTAL_' + label] = currdf_old['matches_' + label].apply(lambda x: x[topic]['total'])
    currdf_old[topic + '_STATS_' + label] = currdf_old['matches_' + label].apply(lambda x: x[topic]['stats'])

  currdf_old.drop(['matches_' + label], axis = 1, inplace = True)
  gc.collect()
  
# Calculate additional stats derived from count stats & sentiment

for label, section in {'FILT_MD': 'MGNT_DISCUSSION', 'FILT_QA': 'QA_SECTION'}.items():
  
 
  for topic in word_set_dict_old.keys():
  
    # relevance = #sentences detected with topic / #total sentences
    currdf_old[topic + '_REL_' + label] = currdf_old[topic + '_TOTAL_' + label].apply(lambda x: len([a for a in x if a>0])/len(x) if len(x)>0 else None)
    currdf_old[topic + '_COUNT_' + label] = currdf_old[topic + '_TOTAL_' + label].apply(lambda x: len([a for a in x if a>0]) if len(x)>0 else None)
    currdf_old[topic + '_SENT_' + label] = currdf_old[[topic + '_TOTAL_' + label, 'SENT_LABELS_' + label]].apply(lambda x: sentscore(x[0], x[1], weight = False), axis = 1)


[                                        ]

*** WARNING: max output size exceeded, skipping output. ***

[########################################] | 100% Completed | 161.20 s


/root/.ipykernel/7698/command-4106393328968512-2900383764:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  currdf_old[topic + '_SENT_' + label] = currdf_old[[topic + '_TOTAL_' + label, 'SENT_LABELS_' + label]].apply(lambda x: sentscore(x[0], x[1], weight = False), axis = 1)
/root/.ipykernel/7698/command-4106393328968512-2900383764:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  currdf_old[topic + '_REL_' + label] = currdf_old[topic + '_TOTAL_' + label].apply(lambda x: len([a for a in x if a>0])/len(x) if len(x)>0 else None)
/

#### refactored

In [0]:
currdf_ref = generate_topic_report(currdf_ref.head(2), word_set_dict_ref, negate_dict1_ref,nlp_ref, stats_list = ['total', 'stats','relevance', 'count', 'sentiment'])


2024-12-13 15:04:52 | INFO    | report_generation.py:228 | Applying initial match count transformations.
2024-12-13 15:04:52 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'FILT_MD' to create 'matches_FILT_MD'.
2024-12-13 15:04:53 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'matches_FILT_MD'.
2024-12-13 15:04:53 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'FILT_QA' to create 'matches_FILT_QA'.
2024-12-13 15:04:53 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'matches_FILT_QA'.
[                                        ] | 0% Completed | 415.79 us2024-12-13 15:04:53 | INFO    | dataframe_utils.py:132 | All transformations applied successfully.
2024-12-13 15:04:53 | DEBUG   | text_preprocessing.py:197 | Tokenized document into 0 tokens.
2024-12-13 15:04:53 | DEBUG   | text_preprocessing.py:197 | Tokenized document into 0 tokens.
2024-12-13 15:04:53 | DEBUG   | t

/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized_nlp_package/data_processing/dataframe_utils.py:115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column] = df[columns_to_use].apply(func)
/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized_nlp_package/data_processing/dataframe_utils.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column] = df.apply(lambda row: func(row), axis=1)
/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized

2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:118 | Applying transformation on multiple columns ['INFLATION_TOTAL_FILT_QA', 'SENT_LABELS_FILT_QA'] to create 'INFLATION_SENT_FILT_QA'.
2024-12-13 15:08:42 | WARNING | text_analysis.py:695 | Indicators and weights must be of the same non-zero length.
2024-12-13 15:08:42 | WARNING | text_analysis.py:695 | Indicators and weights must be of the same non-zero length.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'INFLATION_SENT_FILT_QA'.
2024-12-13 15:08:42 | INFO    | dataframe_utils.py:132 | All transformations applied successfully.
2024-12-13 15:08:42 | INFO    | report_generation.py:246 | Applying transformations for topic 'RESTOCKING' and label 'FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'matches_FILT_QA' to create 'RESTOCKING_TOTAL_FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:127 | Successfully applied 

/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized_nlp_package/data_processing/dataframe_utils.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column] = df.apply(lambda row: func(row), axis=1)
/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized_nlp_package/data_processing/dataframe_utils.py:115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column] = df[columns_to_use].apply(func)
/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized

2024-12-13 15:08:42 | INFO    | report_generation.py:246 | Applying transformations for topic 'LIQUIDITY' and label 'FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'matches_FILT_QA' to create 'LIQUIDITY_TOTAL_FILT_QA'.


...

2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'LIQUIDITY_TOTAL_FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'matches_FILT_QA' to create 'LIQUIDITY_STATS_FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'LIQUIDITY_STATS_FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'LIQUIDITY_TOTAL_FILT_QA' to create 'LIQUIDITY_REL_FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'LIQUIDITY_REL_FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'LIQUIDITY_TOTAL_FILT_QA' to create 'LIQUIDITY_COUNT_FILT_QA'.


/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized_nlp_package/data_processing/dataframe_utils.py:115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column] = df[columns_to_use].apply(func)
/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized_nlp_package/data_processing/dataframe_utils.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column] = df.apply(lambda row: func(row), axis=1)
/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized

2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'LIQUIDITY_COUNT_FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:118 | Applying transformation on multiple columns ['LIQUIDITY_TOTAL_FILT_QA', 'SENT_LABELS_FILT_QA'] to create 'LIQUIDITY_SENT_FILT_QA'.
2024-12-13 15:08:42 | WARNING | text_analysis.py:695 | Indicators and weights must be of the same non-zero length.
2024-12-13 15:08:42 | WARNING | text_analysis.py:695 | Indicators and weights must be of the same non-zero length.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'LIQUIDITY_SENT_FILT_QA'.
2024-12-13 15:08:42 | INFO    | dataframe_utils.py:132 | All transformations applied successfully.
2024-12-13 15:08:42 | INFO    | report_generation.py:246 | Applying transformations for topic 'GEOPOL_UNCERTAINTY' and label 'FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'm

/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized_nlp_package/data_processing/dataframe_utils.py:115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column] = df[columns_to_use].apply(func)
/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized_nlp_package/data_processing/dataframe_utils.py:115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column] = df[columns_to_use].apply(func)
/Workspace/Users/santhosh.kumar3@voya.com/MLFlow_and_NLI_finetune/centralized_nlp_pack

2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'RECESSION_MELTDOWN_BLACK_SWAN_SENT_FILT_QA'.
2024-12-13 15:08:42 | INFO    | dataframe_utils.py:132 | All transformations applied successfully.
2024-12-13 15:08:42 | INFO    | report_generation.py:246 | Applying transformations for topic 'ECONOMIC_DOWNTURN' and label 'FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'matches_FILT_QA' to create 'ECONOMIC_DOWNTURN_TOTAL_FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'ECONOMIC_DOWNTURN_TOTAL_FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:111 | Applying transformation on single column 'matches_FILT_QA' to create 'ECONOMIC_DOWNTURN_STATS_FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataframe_utils.py:127 | Successfully applied transformation for 'ECONOMIC_DOWNTURN_STATS_FILT_QA'.
2024-12-13 15:08:42 | DEBUG   | dataf

In [0]:
assert currdf_old.equals(currdf_ref[currdf_old.columns]), "The DataFrames are not identical."

### Writing Back to Snowflake table 

#### old

In [0]:
from pyspark.sql.types import *

# Auxiliar functions
def equivalent_type(string, f):
    print(string, f)
   # if 'DATE' == string: return StringType()
    
    if f == 'datetime64[ns]': return TimestampType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return FloatType()
    elif 'FILT_MD' == string: return ArrayType(StringType())
    elif 'FILT_QA' == string: return ArrayType(StringType())
    elif '_len_' in string.lower(): return ArrayType(IntegerType())
    elif '_total_' in string.lower(): return ArrayType(IntegerType())
    elif '_count_' in string.lower(): return IntegerType()
    elif '_stats_' in string.lower(): return MapType(StringType(), IntegerType())
    elif 'sent_scores' in string.lower(): return ArrayType(FloatType())
    elif 'sent_labels' in string.lower(): return ArrayType(IntegerType())
 #   elif f == 'object': return ArrayType()
 #   elif f == 'list': return ArrayType()
    else: return StringType()

def define_structure(string, format_type):
    #try: 
    typo = equivalent_type(string, format_type)
    print(typo)
    #except: typo = StringType()
    return StructField(string, typo)

# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark_old(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
      print(column)
    p_schema = StructType(struct_list)
    return spark.createDataFrame(pandas_df, p_schema)                                                                        
# new_sf.db = 'EDS_PROD'
# new_sf.schema = 'QUANT'

In [0]:
for col in ['SENT_LABELS_FILT_MD','SENT_LABELS_FILT_QA']:
  currdf_old[col] = currdf_old[col].apply(ast.literal_eval).astype(object)


spark_parsedDF_old = pandas_to_spark_old(currdf_old)
spark_parsedDF_old = spark_parsedDF_old.replace(np.nan, None)
spark_parsedDF_old = spark_parsedDF_old.withColumn("DATE", F.to_timestamp(spark_parsedDF_old.DATE, 'yyyy-MM-dd'))
spark_parsedDF_old = spark_parsedDF_old.withColumn("PARSED_DATETIME_EASTERN_TZ", F.to_timestamp(spark_parsedDF_old.PARSED_DATETIME_EASTERN_TZ, 'yyyy-MM-dd HH mm ss'))
spark_parsedDF_old = spark_parsedDF_old.withColumn("EVENT_DATETIME_UTC", F.to_timestamp(spark_parsedDF_old.EVENT_DATETIME_UTC, 'yyyy-MM-dd HH mm ss'))   

CALL_ID string
StringType()
CALL_ID
ENTITY_ID string
StringType()
ENTITY_ID
DATE string
StringType()
DATE
FILT_MD string
ArrayType(StringType(), True)
FILT_MD
FILT_QA string
ArrayType(StringType(), True)
FILT_QA
SENT_LABELS_FILT_MD object
ArrayType(IntegerType(), True)
SENT_LABELS_FILT_MD
SENT_LABELS_FILT_QA object
ArrayType(IntegerType(), True)
SENT_LABELS_FILT_QA
CALL_NAME string
StringType()
CALL_NAME
COMPANY_NAME string
StringType()
COMPANY_NAME
EARNINGS_CALL string
StringType()
EARNINGS_CALL
ERROR string
StringType()
ERROR
TRANSCRIPT_STATUS string
StringType()
TRANSCRIPT_STATUS
UPLOAD_DT_UTC datetime64[us]
StringType()
UPLOAD_DT_UTC
VERSION_ID string
StringType()
VERSION_ID
EVENT_DATETIME_UTC datetime64[us]
StringType()
EVENT_DATETIME_UTC
PARSED_DATETIME_EASTERN_TZ datetime64[us]
StringType()
PARSED_DATETIME_EASTERN_TZ
LEN_FILT_MD int64
LongType()
LEN_FILT_MD
LEN_FILT_QA int64
LongType()
LEN_FILT_QA
JOB_CREATION_TOTAL_FILT_MD object
ArrayType(IntegerType(), True)
JOB_CREATION_TOTA

/databricks/spark/python/pyspark/sql/pandas/conversion.py:401: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Expected a string or bytes dtype, got datetime64[us]
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


#### refactored

In [0]:
spark_parsedDF_ref = pandas_to_spark(currdf_ref, column_type_mapping = {'FILT_MD' :  ArrayType(StringType()),
                                                                'FILT_QA' :  ArrayType(StringType()),
                                                                '_len_' :  ArrayType(IntegerType()),
                                                                '_total_' :  ArrayType(IntegerType()),
                                                                '_count_' :  IntegerType(),
                                                                '_stats_' :  MapType(StringType(), IntegerType()),
                                                                'sent_scores' :  ArrayType(FloatType()),
                                                                'sent_labels' :  ArrayType(IntegerType())}, spark = spark)
spark_parsedDF_ref = spark_parsedDF_ref.replace(np.nan, None)

convert_columns_to_timestamp(spark_parsedDF_ref, columns_formats = {'DATE': 'yyyy-MM-dd',
                                                                    'PARSED_DATETIME_EASTERN_TZ': 'yyyy-MM-dd HH mm ss',
                                                                    'EVENT_DATETIME_UTC': 'yyyy-MM-dd HH mm ss'})

In [0]:
spark_parsedDF_old = spark_parsedDF_old.toPandas()
spark_parsedDF_ref = spark_parsedDF_ref.toPandas()

In [0]:
assert spark_parsedDF_ref.equals(spark_parsedDF_old), "The DataFrames are not identical."
